In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# The Knuth-Bendix Completion Algorithm

This notebook presents the algorithm of *ordered completion* for completing a set of equations into a *ground confluent* term rewriting system. It is based on the paper [Ordered Rewriting and Confluence](https://www21.in.tum.de/~nipkow/pubs/cade90.html) by Ursula Martin and Tobias Nipkow.  This notebook is divided into eight sections.
- [Parsing](#Parsing)
- [Matching](#Matching)
- [The Lexicographic Path Ordering](#The-Lexicographic-Path-Ordering)
- [Ordered Term Rewriting](#Ordered-Term-Rewriting)
- [Unification](#Unification)
- [Critical Pairs](#Critical-Pairs)
- [The Completion Algorithm](#The-Completion-Algorithm)
- [Examples](#Examples)

## Parsing

To begin, we need a parser that is capable of parsing terms and equations. This parser is implemented in `Parser.ipynb` and can parse equations of terms and supports the binary operators `+`, `-`, `*`, `/`, `\`, `%`, and `^` (exponentiation) with the usual precedences.  Furthermore, function symbols are supported.  It also provides the function `to_str` for turning terms or equations into strings.  All together, it provides the following functions:
- `parse_file(file_name)` parses the specified file containing equations between terms and returns a list of equations.
- `parse_equation(s)` converts the string `s` into an equation.
- `parse_term(s)` converts the string `s` into a term.
- `to_str(o)` converts an object `o` into a string.  The object `o` can either be
  * a term,
  * an equation,
  * a list of equations,
  * a set of equations, or
  * a dictionary representing a *substitution*.

Terms and equations are represented as nested tuples.  The parser is implemented using the parser generator [Ply](https://www.dabeaz.com/ply/ply.html).

In [ ]:
%run Parser.ipynb

In [ ]:
def test():
    t = parse_term('x * y * z')
    print(t)
    print(to_str(t))
    eq = parse_equation('i(x) * x = 1')
    print(eq)
    print(to_str(parse_file('Examples/group-theory-1.eqn')))
    
test()

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Matching

The function `is_var(t)` checks whether the term `t`is a variable. Variables are represented as nested tuples of the form `($var, name)`, where `name`is the name of the variable.

In [ ]:
def is_var(t):
    return t[0] == '$var'

Given a variable name `x`, the function `make_var(x)` creates a variable with name `x`.

In [ ]:
def make_var(x):
    return ('$var', x)

Given a term `p`, a term `t`,  and a substitution `σ`, the function `match(p, t, σ)` tries to extend the
substitution `σ` so that the equation
$$ p \sigma = t $$
is satisfied.  If this is possible, the function returns `True` and updates the substitution `σ` so that
$p \sigma = t$ holds.  Otherwise, the function returns `False`.  

Substitutions are represented as dictionaries mapping variable names to terms.

In [ ]:
def match(pattern, term, σ={}):
    if is_var(pattern):
        _, var = pattern            # extract variable name
        if var in σ:
            return σ[var] == term
        else:
            σ[var] = term           # extend σ
            return True
    if pattern[0] == term[0] and len(pattern) == len(term):
        return all(match(pattern[i], term[i], σ) for i in range(1, len(pattern)))

In [ ]:
def test():
    p = parse_term('i(x) * z')
    t = parse_term('i(i(y)) * i(y)')
    σ = {}
    match(p, t, σ)
    print(f'σ = {to_str(σ)}')
    σ = {}
    p = parse_term('i(x) * i(x)')
    print(f'match({to_str(p)}, {to_str(t)}, {to_str(σ)}) = {match(p, t, σ)}')
          
test()

Given a term `t` (or a set of terms), the function `find_variables(t)` computes the set of all variable names occurring in `t`.

In [ ]:
def find_variables(t):
    if isinstance(t, set):
        return { var for term in t
                     for var  in find_variables(term)
               }
    if is_var(t):
        _, var = t
        return { var }
    _, *L = t
    return { x for s in L
               for x in find_variables(s)
           }

In [ ]:
def test():
    eq = parse_equation('(x * y) * z = x * (y * z)')
    print(find_variables(eq))
    
test()

Given a term `t` and a substitution `σ` that is represented as a dictionary of the form
$$ \sigma = \{ x_1: s_1, \cdots, x_n:s_n \}, $$
the function `apply(t, σ)` computes the term that results from replacing the variables $x_i$ with the terms $s_i$ in `t` for all $i=1,\cdots,n$.  This term is written as $t\sigma$ and if $\sigma = \{ x_1: s_1, \cdots, x_n:s_n \}$, then $t\sigma$ is defined by induction on `t` as follows:
- $x_i\sigma := s_i$,
- $v\sigma := v$ if $v$ is a variable and $v \not\in \{x_1,\cdots,x_n\}$,
- $f(t_1,\cdots,t_n)\sigma := f(t_1\sigma, \cdots, t_n\sigma)$.

In [ ]:
def apply(t, σ):
    "Apply the substitution σ to the term t."
    if is_var(t):
        _, var = t
        if var in σ:
            return σ[var]
        else:
            return t
    else: 
        f, *Ts = t
        return (f,) + tuple(apply(s, σ) for s in Ts)

In [ ]:
def test():
    p = parse_term('i(x) * x')
    t = parse_term('i(i(y)) * i(y)')
    σ = {}
    match(p, t, σ)
    print(to_str(apply(p, σ)))
    
test()

Given a set of terms or equations `Ts` and a substitution `σ`, the function `apply_set(T, σ)` applies the substitution `σ` to all elements in `Ts`.

In [ ]:
def apply_set(Ts, σ):
    return { apply(t, σ) for t in Ts }

If  $\sigma = \big[ x_1 \mapsto s_1, \cdots, x_m \mapsto s_m \big]$ and
$\tau = \big[ y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big]$ 
are two substitutions that are *non-overlapping*, i.e. such that $\{x_1,\cdots, x_m\} \cap \{y_1,\cdots,y_n\} = \{\}$ holds,
then we define the *composition* $\sigma\tau$ of $\sigma$ and $\tau$ as follows:
$$\sigma\tau := \big[ x_1 \mapsto s_1\tau, \cdots, x_m \mapsto s_m\tau,\; y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big]$$
This definition implies that the following *associative law* is valid:
$$ x(\sigma\tau) = (x\sigma)\tau $$
The function $\texttt{compose}(\sigma, \tau)$ takes two non-overlapping substitutions and computes their composition.

In [ ]:
def compose(σ, τ):
    Result = { x: apply(s, τ) for (x, s) in σ.items() }
    Result.update(τ)
    return Result

In [ ]:
def test():
    t1 = parse_term('i(y)')
    t2 = parse_term('a * b')
    t3 = parse_term('i(b)')
    σ = { 'x': t1 }
    τ = { 'y': t2, 'z': t3  }
    print(f'compose({to_str(σ)}, {to_str(τ)}) = {to_str(compose(σ, τ))}')
    
test()  

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## The Lexicographic Path Ordering

In order to use an equation $s = t$ as a rewrite rules, we have to check whether $s$ is more complex that $t$, so that $s$ should be simplified to $t$, or whether $t$ is more complex than $s$ and we should rewrite $t$ into $s$.  To this end, we define the *lexicographic path ordering*, which is an ordering on the set $\mathcal{T}(\Sigma,\mathcal{V})$ of $\Sigma$-terms with variables from the set $\mathcal{V}$.  In order to define the lexicographic path ordering, we assume that a total order $\leq$ on the set $\Sigma$ of function symbols is given.  The notation $f < g$ is used as an abbreviation for the formula $f \leq g \wedge f \not= g$.  

Let $s,t\in\mathcal{T}(\Sigma,\mathcal{V})$.  Then we define $v \prec t$ by induction on the structure of $s$ and $t$ as follows:
1. $v \prec t$ if $v$ is a variable occurring in $t$ and $t \not= v$.
2. $f(s_1,\cdots,s_m) \prec g(t_1,\cdots,t_n)$ if the following holds:
   * $f < g$ and 
   * $s_i \prec g(t_1,\cdots,t_n)$ for all $i=1,\cdots, m$.
3. $f(s_1,\cdots,s_m) \prec f(t_1,\cdots,t_m)$ if 
   * there exists an $i \in \{1,\cdots,n\}$ such that $f(s_1,\cdots,s_m) \preceq t_i$ or
   * both of the following conditions are true:
     - $s_i \prec f(t_1,\cdots,t_m)$ for all $i=1,\cdots, m$ and
     - $[s_1, \cdots, s_m] \prec_{\textrm{lex}} [t_1, \cdots, t_m]$.
   
   Here, $\prec_{\textrm{lex}}$ denotes the *lexicographic extension* of the ordering $\prec$ to
   lists of terms.  It is defined as follows:
   $$ [x] + R_1 \prec_{\textrm{lex}} [y] + R_2 \;\stackrel{_\textrm{def}}{\Longleftrightarrow}\;
      x \prec y \,\vee\, \bigl(x = y \wedge R_1 \prec_{\textrm{lex}} R_2\bigr)
   $$
4. $f(s_1,\cdots,s_m) \prec g(t_1,\cdots,t_n)$ if
   * $f > g$ and 
   * there exists an $i \in \{1,\cdots,n\}$ such that $f(s_1,\cdots,s_m) \preceq t_i$.

We define $s \preceq t \;\stackrel{_\textrm{def}}{\Longleftrightarrow}\; s \prec t \vee s = t$.

Given a variable name `x` and a term `t`, the function `occurs(x, t)` checks whether `x` occurs in `t`.

In [ ]:
def occurs(x, t):
    if is_var(t):
        _, var = t
        return x == var
    return any(occurs(x, arg) for arg in t[1:])

The total order $\leq$ on the set of function symbols is defined via the dictionary `Level`given below.  We have
$$ f < g \;\stackrel{_\textrm{def}}{\Longleftrightarrow}\; \texttt{Level}[f] < \texttt{Level}[g]. $$

In [ ]:
Level = { '0': 0, '1': 1, '+': 2, '*': 3, 'i': 4 }

The function `is_simpler(s, t, Ordering)` receives three arguments.
- `s` and `t` are terms.
- `Ordering` is a dictionary mapping variable names to <u>different</u> natural numbers.
  Those variable names in the domain of `Ordering` are interpreted as constant symbols.
  On these constant symbols a total order is defined via the dictionary `Ordering`: 
  $$ x < y \;\stackrel{_\textrm{def}}{\Longleftrightarrow}\; \texttt{Ordering}[x] < \texttt{Ordering}[y]. $$

In [ ]:
def is_simpler(s, t, Ordering=None):
    if is_var(s):
        _, x = s
        if s != t and occurs(x, t):            
            return True
        elif Ordering != None:  # x is taken as a constant
            if is_var(t):       
                _, y = t
                if x not in Ordering:
                    return False
                if y not in Ordering:
                    return True
                else:
                    return Ordering[x] < Ordering[y]
            else:
                return True    # variables are smaller than function symbols
        else:                              
            return False
    if is_var(t):
        return False
    f, *sArgs = s
    g, *tArgs = t
    if Level[f] < Level[g]:
        return all(is_simpler(arg, t, Ordering) for arg in sArgs)
    if f == g:
        assert len(sArgs) == len(tArgs)
        return any(s == arg or is_simpler(s, arg, Ordering) for arg in tArgs) or \
               all(is_simpler(arg, t, Ordering) for arg in sArgs) and \
               is_simpler_list(sArgs, tArgs, Ordering)
    if Level[f] > Level[g]:
        return any(s == arg or is_simpler(s, arg, Ordering) for arg in tArgs)

Given two lists `S` and `T` of terms and a dictionary `D`, the function `is_simpler_list(S, T, D)` checks whether `S` is lexicographically simpler than `T` if the elements of `S` and `T` are compared with the *lexicographical path ordering* $\prec$.  It is assumed that `S` and `T` have the same length.

In [ ]:
def is_simpler_list(S, T, Ordering=None):
    if S == [] == T:
        return False
    if is_simpler(S[0], T[0], Ordering):
        return True
    if S[0] == T[0]:
        return is_simpler_list(S[1:], T[1:], Ordering)
    return False

In [ ]:
def test():
    s = parse_term('a * (b + c)')
    t = parse_term('a * b + a * c')
    O = { 'a': 0, 'b': 1, 'c': 2 }
    print(is_simpler(s, t, O))
    print(is_simpler(t, s, O))
    
test()

In [ ]:
def test():
    Ordering = { 'x': 3, 'y': 2, 'z': 1 }
    _, l, r = parse_equation('(x * y) * z = x * (y * z)')
    print(is_simpler(r, l))
    print(is_simpler(r, l, Ordering))
    Ordering = { 'x': 1, 'y': 2, 'z': 3 }
    print(is_simpler(r, l, Ordering))
        
test()

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Ordered Term Rewriting

In [ ]:
from string import ascii_lowercase
ascii_lowercase

Given a term `s` and a set of variables `V`, the function `rename_variables(s, V)` renames the variables in `s` so that they differ from the variables in the set `V`.  This will only work if the number of variables occurring in `V` times two is less than the number of letters in the latin alphabet, i.e. less than 26.  Therefore, the set `V` must have at most 13 variables.  For our examples, this is not a restriction.

In [ ]:
def rename_variables(s, Vars):
    assert len(Vars) <= 13, f'Error: too many variables in {Vars}.'
    NewVars = set(ascii_lowercase) - Vars
    NewVars = sorted(list(NewVars))
    σ       = { x: make_var(NewVars[i]) for (i, x) in enumerate(Vars) }
    return apply(s, σ)

In [ ]:
def test():
    Vars = { 'x', 'y', 'z' }
    t = parse_term('x * y * z')
    print(f'rename_variables({to_str(t)}, {Vars}) = {to_str(rename_variables(t, Vars))}')
    
test()

The function `simplify_step(t, E, O)` takes three arguments:
- `t` is a term,
- `E` is a set of equations of the form `('=', l, r)`,
- `O` is an ordering of the variables occurring in `t`.

It is assumed that the term `t` does not share variables with the set `E`.

The function tries to find an equation `l = r` in `E` and a subterm `s` in the term `t` such that the left hand side `l` of the equation matches the subterm `s` using some substitution $\sigma$, i.e. we have $s = l\sigma$.  Then the term `t` is simplified by replacing the subterm `s` in `t` by $r\sigma$ provided that $r\sigma \prec l\sigma$.  More formally, if `u` is the position of `s` in `t`, i.e. `t/u = s` then `t` is simplified into the term 
$$ t[u \mapsto r\sigma]. $$
If an appropriate subterm `s` is found, the simplified term $t[u \mapsto r\sigma]$ is returned.  Otherwise, the function returns `None`.

In [ ]:
def simplify_step(t, Equations, Ordering=None):
    if is_var(t):
        return None
    for eq in Equations:
        _, lhs, rhs = eq
        σ = {}
        if match(lhs, t, σ):
            lhsσ = apply(lhs, σ)
            rhsσ = apply(rhs, σ)
            if is_simpler(rhsσ, lhsσ, Ordering):
                return rhsσ
    f, *Args   = t
    simpleArgs = []
    change     = False
    for arg in Args:
        simple = simplify_step(arg, Equations, Ordering)
        if simple != None:
            change = True
            simpleArgs += [simple]
        else:
            simpleArgs += [arg]
    if change:
        return (f,) + tuple(simpleArgs)
    return None

In [ ]:
def test():
    E = { parse_equation('(x * y) * z = x * (y * z)') }
    O = { 'a': 1, 'b': 2, 'c': 3 }
    t = parse_term('(a * c) * b')
    print(to_str(simplify_step(t, E)))
    print(to_str(simplify_step(t, E, O)))
    
test()

The function `normal_form(t, E, O)` takes a term `t`, a list (or set) of equations `E`, and a variable `Ordering` for the variables occurring in `t` and tries to simplify the term `t` as much as possible using the equations from `E`.  

In the implementation, we have to be careful to rename the variables occurring in `E` so that they are different from the variables occurring in `t`.  Furthermore, we have to take care that we don't identify different variables in `E` by accident.  Therefore, we rename the variables in `E` so that they are both different from the variables in `t` and from the old variables occurring in `E`.

In [ ]:
def normal_form(t, E, Ordering=None):
    Vars_t = find_variables(t)
    NewE = []
    for eq in E:
        Vars_eq = find_variables(eq)
        NewE += [ rename_variables(eq, Vars_t | Vars_eq) ]   
    while True:
        s = simplify_step(t, NewE, Ordering)
        if s == None:
            return t
        t = s

In [ ]:
def test():
    Ordering = { 'a': 1, 'b': 2, 'c': 3 }
    s   = parse_term('(c * a) * b')
    eq1 = parse_equation('x * y = y * x')
    eq2 = parse_equation('(x * y) * z = x * (y * z)')
    eq3 = parse_equation('x * (y * z) = y * (x * z)')
    print(to_str(normal_form(s, { eq1, eq2, eq3 }, Ordering)))

    test()

In [ ]:
def test():
    O = { 'x': 1, 'y': 2, 'z': 3 }
    E = parse_file('Examples/group-theory-1.eqn')
    t = parse_term('(x * i(y)) * y * z')
    print(to_str(normal_form(t, E)))
    
test()

Given an equation `eq`, a set of `RewriteRules`, and an `Ordering` of the variables occurring in `eq`, the function `simplify_equation` simplifies the equation `eq` using the given rewrite rules. 

In [ ]:
def simplify_equation(eq, RewriteRules, Ordering=None):   
    _, s, t = eq
    new_s = normal_form(s, RewriteRules, Ordering)
    new_t = normal_form(t, RewriteRules, Ordering)
    return ('=', new_s, new_t)

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Unification

In this section, we implement the [unification](https://en.wikipedia.org/wiki/Unification_(computer_science)) algorithm of *Martelli and Montanari*.

The algorithm implemented below takes a pair `(E, σ)` as its input.  Here `E` is a set of *syntactical equations* that are to be solved and `σ` is a substitution that is initially empty.  The pair `(E, σ)` is then transformed using the rules of Martelli and Montanari.  The transformation is successful if the pair `(E, σ)` can be transformed into a pair of the form `({}, μ)`.  Then `μ` is the *solution* to the system of equations `E`. 
The rules that can be used to solve a system of *syntactical equations* are as follows:
- If $y\in\mathcal{V}$ is a variable that does **not** occur in the term $t$,
     then we perform the following reduction: 
     $$ \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle \quad\leadsto \quad 
         \Big\langle E[y \mapsto t], \sigma\big[ y \mapsto t \big] \Big\rangle 
     $$
      
- If the variable $y$ occurs in the term $t$, then the system of syntactical equations
     $E \cup \big\{ y \doteq t \big\}$ is not solvable:
     $$ \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle\;\leadsto\; \texttt{None} \quad
        \mbox{if $y \in \textrm{Var}(t)$ and $y \not=t$.}$$

- If $y\in\mathcal{V}$ is a variable and $t$ is no variable, then we use the following rule:
     $$ \Big\langle E \cup \big\{ t \doteq y \big\}, \sigma \Big\rangle \quad\leadsto \quad 
         \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle.
     $$   

- Trivial syntactical equations of variables can be dropped:
     $$ \Big\langle E \cup \big\{ x \doteq x \big\}, \sigma \Big\rangle \quad\leadsto \quad
         \Big\langle E, \sigma \Big\rangle.
     $$   

- If $f$ is an $n$-ary function symbol, then we have: 
     $$ \Big\langle E \cup \big\{ f(s_1,\cdots,s_n) \doteq f(t_1,\cdots,t_n) \big\}, \sigma \Big\rangle 
         \;\leadsto\; 
         \Big\langle E \cup \big\{ s_1 \doteq t_1, \cdots, s_n \doteq t_n\}, \sigma \Big\rangle.
     $$   

- The system of syntactical equations $E \cup \big\{ f(s_1,\cdots,s_m) \doteq g(t_1,\cdots,t_n) \big\}$
     has **no** solution if the function symbols $f$ and $g$ are different:
     $$ \Big\langle E \cup \big\{ f(s_1,\cdots,s_m) \doteq g(t_1,\cdots,t_n) \big\},
      \sigma \Big\rangle \;\leadsto\; \texttt{None} \qquad \mbox{if $f \not= g$}.
     $$
</ol>


Given two terms $s$ and $t$, the function $\texttt{unify}(s, t)$ computes the <em style="color:blue;">most general unifier</em> of $s$ and $t$.

In [ ]:
def unify(s, t):
    return solve({ ('≐', s, t) }, {})

Given a set of <em style="color:blue;">syntactical equations</em> $E$ and a substitution $\sigma$, the function $\texttt{solve}(E, \sigma)$ applies the rules of Martelli and Montanari to solve $E$.

In [ ]:
def solve(E, σ):
    while E != set():
        _, s, t = E.pop()
        if s == t: # remove trivial equations
            continue
        if is_var(s):
            _, x = s
            if occurs(x, t):
                return None
            else: # set x to t
                E = apply_set(E, { x: t })
                σ = compose(σ, { x: t })
        elif is_var(t):
            E.add(('≐', t, s))
        else:
            f    , g     = s[0]      , t[0]
            sArgs, tArgs = s[1:]     , t[1:]
            m    , n     = len(sArgs), len(tArgs)
            if f != g or m != n:
                return None
            else:
                E |= { ('≐', sArgs[i], tArgs[i]) for i in range(m) }
    return σ

In [ ]:
def test():
    s = parse_term('x * i(x) * (y * z)')
    t = parse_term('a * i(1) * b')
    print(f'unify({to_str(s)}, {to_str(t)}) = {to_str(unify(s, t))}')
    
test()

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## Critical Pairs

The central idea is the notion of a *critical pair*.  

Given two equations `lhs1 = rhs1` and `lhs2 = rhs2`, a pair of terms `(s, t)` is a *critical pair* of these equations if we have the following:
- `u` is a non-trivial position in `lhs1`, i.e. $u \in \mathcal{P}os(\texttt{lhs1})$ and `lhs1/u` is not a variable,
- The subterm `lhs1/u` is unifiable with `lhs2`, i.e.
  $$\mu = \texttt{mgu}(\texttt{lhs}_1 / a, \texttt{lhs}_2) \not= \texttt{None},$$
- $s = \texttt{lhs}_1[u \leftarrow \texttt{rhs}_2]\mu$ and $t = \texttt{rhs}_1\mu$. 

The function `critical_pairs` implemented in this section computes the set of all critical pairs between two equations. 

Given a term `t`, the function `non_triv_positions` computes the set $\mathcal{P}os(t)$ of all *positions* in `t` that do not point to variables.  Such positions are called *non-trivial positions*.  Given a term `t`, the set $\mathcal{P}os(t)$ of all positions in $t$ is defined by induction on `t`. 
1. $\mathcal{P}os(v) := \bigl\{()\bigr\} \quad \mbox{if $v$ is a variable} $
2. $\mathcal{P}os\bigl(f(t_0,\cdots,t_{n-1})\bigr) := 
     \bigl\{()\bigr\} \cup 
     \bigl\{ (i,) + u \mid i \in\{0,\cdots,n-1\} \wedge u \in \mathcal{P}os(t_i) \bigr\}
   $
   
Note that since we are programming in Python, positions are zero-based.  Given a position $u$ in a term $t$, we define $t/u$ as the *subterm of $t$ at position $u$* by induction on $t$:
1. $t/() := t$,
2. $f(t_0,\cdots,t_{n-1})/u := t_i/u\texttt{[1:]}$.

Given a term $s$, a term $t$, and a position $u \in \mathcal{P}os(t)$, we also define the *replacement of the subterm at position $u$ by $t$*, written $s[u \mapsto t]$ by induction on $u$:
1. $s\bigl[() \mapsto t\bigr] := t$.
2. $f(s_0,\cdots,s_{n-1})\bigl[\bigl((i,) + u\bigr) \mapsto t\bigr] := f\bigl(s_0,\cdots,s_i[u \mapsto t],\cdots,s_{n-1}\bigr)$.

In [ ]:
def non_triv_positions(t):
    if is_var(t):
        return set()   # compute only non-trivial positions 
    _, *args = t
    Result = { () }
    for i, arg in enumerate(args):
        Result |= { (i,) + a for a in non_triv_positions(arg) }
    return Result

In [ ]:
def test():
    t = parse_term('x * i(x) * (a * 1)')
    print(f'non_triv_positions({to_str(t)}) = {non_triv_positions(t)}')
    
test()

Given a term `t` and a position `u` in `t`, the function `subterm(t, u)` extracts the subterm that is located at position `u`, i.e. it computes `t/u`. The position `u` is zero-based.

In [ ]:
def subterm(t, u):
    if len(u) == 0:
        return t
    _, *args = t
    i, *ru   = u
    return subterm(args[i], ru)

In [ ]:
def test():
    t = parse_term('x * i(x) * 1')
    print(f'subterm({to_str(t)}, (0,1)) = {to_str(subterm(t, (0,1)))}')
    
test()

Given a term `t`, a position `u` in `t` and a term `s`, the function `replace_at(t, u, s)` replaces the subterm at position `u` with `t`, i.e. it returns the term $s[u \mapsto t]$ at the beginning of this section.

In [ ]:
def replace_at(t, u, s):
    if len(u) == 0:  # note that u is either a list or a tuple
        return s
    i, *Ur   = u
    f, *Args = t
    NewArgs  = []
    for j, arg in enumerate(Args):
        if j == i:
            NewArgs.append(replace_at(arg, Ur, s))
        else:
            NewArgs.append(arg)
    return (f,) + tuple(NewArgs)

In [ ]:
def test():
    t = parse_term('x * i(x) * 1')
    s = parse_term('a * b') 
    print(f'replace_at({to_str(t)}, (0,1), {to_str(s)}) = {to_str(replace_at(t, (0,1), s))}')
    
test()

Given two equations `eq1` and `eq2`, the function `critical_pairs(eq1, eq2)` computes the set of all *critical pairs* between these equations.  A pair of terms `(s, t)` is a *critical pair* of `eq1` and `eq2` iff
- `eq1` has the form `lhs1 = rhs1`,
- `eq2` has the form `lhs2 = rhs2`,
- `u` is a non-trivial position in `lhs1`,
- $\mu = \texttt{mgu}(\texttt{lhs}_1/u, \texttt{lhs}_2) \not= \texttt{None}$,
- $s = \texttt{lhs}_1[u \leftarrow \texttt{rhs}_2]\mu$ and $t = \texttt{rhs}_1\mu$. 

In [ ]:
def critical_pairs(eq1, eq2):
    Vars = find_variables(eq1) | find_variables(eq2)  # prevent accidental name clashes 
    eq2  = rename_variables(eq2, Vars) 
    _, lhs1, rhs1 = eq1
    _, lhs2, rhs2 = eq2
    Result = set()
    Positions = non_triv_positions(lhs1)
    for u in Positions:
        s = subterm(lhs1, u)
        𝜇 = unify(s, lhs2)
        if 𝜇 != None:
            s = replace_at(lhs1, u, rhs2)
            s = apply(s, 𝜇)
            t = apply(rhs1, 𝜇)
            if s != t:
                Result.add( (s, t) )
    return Result

In [ ]:
def test():
    eq1 = parse_equation('i(x) * x = 1')
    eq2 = parse_equation('(x * y) * z = x * (y * z)')
    for s, t in critical_pairs(eq2, eq1):
        print(f'{to_str(s)} = {to_str(t)}')
        
test()

[Back to top](#The-Knuth-Bendix-Completion-Algorithm)

## The Completion Algorithm

The function `print_equations` prints the set of `Equations` one by one and numbers them.

In [ ]:
def print_equations(Equations):
    cnt = 1
    for _, l, r in Equations:
        print(f'{cnt}. {to_str(l)} = {to_str(r)}')
        cnt += 1

In [ ]:
def test():
    Equations = parse_file('Examples/group-theory-1.eqn')
    print_equations(Equations)
    
test()

Given an equation `eq` of the form `eq = ('=', lhs, rhs)`, the function `complexity(eq)` computes a measure of complexity for the given equation.   This measure of complexity is the length of the string that represents the equation.  This measure of complexity is later used as the priority in the priority queue that stores equations. Less complex equations are more interesting and should be considered first when computing critical pairs.

In [ ]:
def complexity(eq):
    return len(to_str(eq))

In [ ]:
def test():
    eq = parse_equation('x * i(x) = 1')
    print(complexity(eq))
    
test()

Given a set of equations `RewriteRules` and a single rewrite rule `eq`, the function `all_critical_pairs(RewriteRules, eq)` computes the set of all *critical pairs* that can be build from an equation in `RewriteRules` and the equation `eq`.  It is assumed that `eq` is already an element of `RewriteRules`.  

In [ ]:
def all_critical_pairs(RewriteRules, eq):
    Result = set()
    for eq1 in RewriteRules:
        Result |= { (('=', l, r), eq1, eq) for l, r in critical_pairs(eq1, eq) }
        Result |= { (('=', l, r), eq, eq1) for l, r in critical_pairs(eq, eq1) }
    return Result

In [ ]:
def test():
    E  = parse_file('Examples/group-theory-1.eqn')
    eq = parse_equation('(x * y) * z = x * (y * z)')   
    for cp, eq1, eq2 in all_critical_pairs(E, eq):
        print(f'{to_str(cp)} from {to_str(eq1)} and {to_str(eq2)}')
        
test()

The module `heapq` provides heap-based priority queues, which are implemented as lists.

In [ ]:
import heapq as hq

Given an equation `eq` of the form `('=', s, t)` and a priority queue of equations `EquationQueue`, the function `enqueue_equation(EquationQueue, eq)` inserts the equations `('=', s, t)` and `('=', t, s)` into the priority queue, provided any of the respective equation can be used for ordered rewriting.

In [ ]:
def enqueue_equation(EquationQueue, eq):
    _, s, t = eq
    if s == t:
        return
    priority = complexity( ('=', s, t) )
    if not is_simpler(s, t):
        hq.heappush(EquationQueue, (priority, ('=', s, t)) )
    if not is_simpler(t, s):
        hq.heappush(EquationQueue, (priority, ('=', t, s)) )

In [ ]:
def test():
    Queue = []
    eq1   = parse_equation('x * (y * z) = (x * y) * z')
    eq2   = parse_equation('x * y = y * x')
    enqueue_equation(Queue, eq1)
    enqueue_equation(Queue, eq2)
    while Queue != []:
        _, rule = hq.heappop(Queue)
        print(to_str(rule))
        
test()

Given a set `M` of elements, the function `all_permutations` computes the set of all permutations of `M`.

In [ ]:
def all_permutations(M):
    n = len(M)
    if n == 0:
        return { () }
    x = M.pop() 
    return { P[:i] + (x,) + P[i:] for P in all_permutations(M)
                                  for i in range(n)
           }

In [ ]:
all_permutations({1,2,3})

Given a set `V` of variables, the function `all_orderings` computes a list of all possible *orderings* of these variables.  These orderings are represented as dictionaries.  If an ordering is encoded as the dictionary `O` and $x$ and $y$ are variables, then we have 
$$ x < y \;\stackrel{_\textrm{def}}{\Longleftrightarrow}\; O[x] < O[y]. $$

In [ ]:
def all_orderings(V):
    return [ { x: i for i, x in enumerate(P) } for P in all_permutations(V) ]

In [ ]:
all_orderings({'x', 'y', 'z'})

In [ ]:
def is_instance(eq, RewriteRules):
    Variables_eq = find_variables(eq)
    for rule in RewriteRules:
        Variables_rule = find_variables(rule)
        rule = rename_variables(rule, Variables_eq | Variables_rule)
        if match(rule, eq, {}):
            return True
    return False

Given an equation `eq` and a set of `RewriteRules`, the function `is_trivial(eq, RewriteRules)` checks whether the equation `eq` can be rewritten into a trivial equation of the form `s = s` for all possible orderings of the variables occurring in `eq`.

In [ ]:
def is_trivial(eq, RewriteRules):
    if is_instance(eq, RewriteRules):
        return True
    Variables = find_variables(eq)
    for Ordering in all_orderings(Variables):
        simple = simplify_equation(eq, RewriteRules, Ordering)
        if simple[1] != simple[2]:
            return False
    return True

In [ ]:
def test():
    eq  = parse_equation('(a + b) + z = z + (a + b)')
    eq1 = parse_equation('x + y = y + x')
    eq2 = parse_equation('(x + y) + z = x + (y + z)')
    eq3 = parse_equation('x + (y + z) = y + (x + z)')
    return is_trivial(eq, { eq1, eq2, eq3 })

test()

Given a priority queue of equations `EquationQueue`, a set of `RewriteRules`, and a set of `CriticalPairs`, the function `enqueue_critical_pairs(EquationQueue, RewriteRules, CriticalPairs)` inserts the critical pairs that can not be reduced to trivial equations for all orderings into the queue `EquationQueue`.

In [ ]:
def enqueue_critical_pairs(EquationQueue, RewriteRules, CriticalPairs):
    for cp, eq1, eq2 in CriticalPairs:
        cp = simplify_equation(cp, RewriteRules)
        if cp[1] != cp[2] and not is_trivial(cp, RewriteRules):
            # print(f'cp:      {to_str(cp)} from {to_str(eq1)}, {to_str(eq2)}')
            enqueue_equation(EquationQueue, cp)

Given a priority queue `EquationQueue`, a set of `RewriteRules`, and a new rewrite `rule`, the function `simplify_rules(EquationQueue, RewriteRules, rule)` tries to simplify all rules in `RewriteRules` with `rule`.  If an equation `eq` from `RewriteRules` can be simplified with `rule`, it is further simplified with all rules in `RewriteRules`.  After that, it is put back into the `EquationQueue`.  The set of those rewrite rules that have not been simplified is returned.

In [ ]:
def simplify_rules(EquationQueue, RewriteRules, rule):
    SimpleEqs = set()
    for eq in RewriteRules:
        _, s, t = eq
        new_s = normal_form(s, { rule })
        if new_s != s:
            new_s = normal_form(new_s, RewriteRules | { rule })
        new_t = normal_form(t, { rule })
        if new_t != t:
            new_t = normal_form(new_t, RewriteRules | { rule })
        new_eq = ('=', new_s, new_t)
        if new_eq == eq:
            SimpleEqs.add(eq)
        elif new_s != new_t:
            print(f'old:     {to_str(eq)}')
            print(f'new:     {to_str(new_eq)}')      
            enqueue_equation(EquationQueue, new_eq)
        else:
            print(f'removed: {to_str(eq)}')    
    return SimpleEqs    

Given a set of `RewriteRules`, the function `remove_trivial_rules(RewriteRules)` returns the set of those rules that are not redundant. 

In [ ]:
def remove_trivial_rules(RewriteRules):
    Rules = set()
    for rule in RewriteRules:
        if not is_trivial(rule, RewriteRules - { rule }):
            Rules.add(rule)
    return Rules

Given a file name that contains a set of equations and a dictionary encoding an ordering of the function symbols, the function `completion_algorithm` tries to complete the set of equations:
1. The given equations are ordered.
2. The ordered equations are pushed into the priority queue `EquationQueue` according to their complexity.
3. The set `RewriteRules` is initialized as the empty set.
4. As long as the priority queue is not empty, the least complex equation `lr` is removed from the 
   priority queue and simplified using the known `RewriteRules`.  
5. If the simplified version of `lr` is non-trivial, all critical pairs between it and the 
   `RewriteRules` are computed.  These critical pairs are pushed onto the priority queue.
6. When no new critical pairs can be found, the set of `RewriteRules` is returned.
   This set is then guaranteed to be a *ground confluent* set of rewrite rules.

In [ ]:
def completion_algorithm(file):
    Axioms        = parse_file(file)
    EquationQueue = []
    for eq in Axioms:
        print(f'given:   {to_str(eq)}')
        enqueue_equation(EquationQueue, eq)
    RewriteRules = set()
    while EquationQueue != []:
        _, eq     = hq.heappop(EquationQueue)
        eq_simple = simplify_equation(eq, RewriteRules)
        if eq != eq_simple:
            enqueue_equation(EquationQueue, eq_simple)
            continue
        if eq[1] != eq[2] and not is_trivial(eq, RewriteRules):
            print(f'added:   {to_str(eq)}')
            CriticalPairs = all_critical_pairs(RewriteRules | { eq }, eq)
            enqueue_critical_pairs(EquationQueue, RewriteRules, CriticalPairs)
            RewriteRules = simplify_rules(EquationQueue, RewriteRules, eq)
            RewriteRules.add(eq)
    RewriteRules = remove_trivial_rules(RewriteRules)
    print()
    print_equations(RewriteRules)
    return RewriteRules

## Examples 

### Groups

A structure $\mathcal{G} = \langle G, 1, *, i \rangle$ is a [group](https://en.wikipedia.org/wiki/Group_(mathematics)) iff
1. $G$ is a set.
2. $1 \in G$.
3. $*: G \times G \rightarrow G$,

   where $*$ is called the *multiplication* of $\mathcal{G}$.
4. $i: G \rightarrow G$
5. The following equations hold for all $x,y,z \in G$:
   * $1 * x = x$, i.e. $1$ is a *left-neutral element*.
   * $i(x) * x = 1$, i.e. $i(x)$ is a *left-inverse* of $x$.
   * $(x * y) * z = x * (y * z)$, i.e. the multiplication is *associative*.

A typical example of a group is the set of invertible $n \times n$ matrices.

Given the axioms defining a group, the *Knuth-Bendix completion* algorithm is able to prove the following:
1. The *left neutral element* is also a *right neutral element*, we have:
   $$ x * 1 = x $$
2. The *left inverse* is also a *right inverse*, we have:
   $$ x * i(x) = 1 $$
3. The operations $i$ and $*$ commute as follows:
   $$ i(x * y) = i(y) * i(x) $$

In [ ]:
!cat Examples/group-theory-1.eqn

In [ ]:
%%time
RewriteRules = completion_algorithm('Examples/group-theory-1.eqn')

### Groups of Exponent Two

A structure $\mathcal{G} = \langle G, 1, *, i \rangle$ is a *group of exponent two* iff $\mathcal{G}$ is a group and, furthermore, the equation $x * x = 1$ holds for all $x$.  Note that in groups of exponent two the commutativity law $x * y = y * x$ is valid.

In [ ]:
!cat Examples/groups-of-exponent-two.eqn

In [ ]:
%%time
RewriteRules = completion_algorithm('Examples/groups-of-exponent-two.eqn')

### Boolean Rings

In [ ]:
!cat -n Examples/boolean-rings.eqn

In [ ]:
%%time
RewriteRules = completion_algorithm('Examples/boolean-rings.eqn')

### Commutative Groups

For *commutative groups*, i.e. for groups that satisfy the *commutativity law* $x * y = y * x$ there is no finite set of rewrite rules that is confluent.  The problem is that the algorithm generates equations of the form
$$ y * (x_1 * (x_2 * \cdots * (i(y) * x_n)\cdots) = x_1 * (x_2 * \cdots * x_n)\cdots) $$
for all $n \in \mathbb{N}$ and these equations can not be reduced to a common normal form.

In [ ]:
%%time
RewriteRules = completion_algorithm('Examples/commutative-groups.eqn')